In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
with open("/content/drive/My Drive/tweets_corpus.txt",mode = 'r') as f:
  file_text = f.read()
# lines = file_text.readline()

In [0]:
lines = file_text.split('\n')

In [4]:
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer 
import collections
from collections import OrderedDict
ord_dict = collections.OrderedDict()
lemmatizer = WordNetLemmatizer() 
  
nltk.download('stopwords')

stop_words = set(stopwords.words('english')) 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
import re
from nltk.stem import PorterStemmer

In [0]:
doc_ids = []
inverted_index = {}
total_text = ''
ps = PorterStemmer()
no_of_documents = 0
word_count = []
for line in lines:
  ## split Doc id and tweet 
  tweet = line.split('\t')
  
  if len(tweet) > 1:
    no_of_documents += 1
    doc_ids.append(tweet[0])
    ## lowercase the string 
    text_lower = tweet[1].lower()
    ## take only words
    text_lower = re.sub(r'[^\w]', ' ', text_lower)
    ## split the text 
    word_splt = text_lower.split()
    ## tokenize the word 
    word_tokeniz = []
    for word in word_splt:
       word_tokeniz.append(word) 
    ## Remove stop words
    wrd_stp = [w for w in word_tokeniz if not w in stop_words]
    ## Lemmatize word
    word_count.append(len(wrd_stp))
    for word in wrd_stp:
      stem_word = ps.stem(word)
      ## Only Take words > 2 length 
      if len(stem_word) > 1:
        if stem_word in inverted_index:
          ## create a dictionary with key as word and value as document id 
          inverted_index[stem_word].append(tweet[0])
        else:
          inverted_index[stem_word] = [tweet[0]]
      

In [0]:
from copy import deepcopy as deepcopy
inverted_index1 = deepcopy(inverted_index)

In [0]:
for key,value in inverted_index.items():
  # if key == 'chocol':
  #   print(inverted_index1[key])
  inverted_index[key] = sorted(list(set(value)))

In [0]:
import math
import numpy as np
tf_idf_word = {}
for word in list(inverted_index.keys()):
  tf_idf_word[word] = np.zeros((no_of_documents,))
  for i,doc_id in enumerate(doc_ids):
    word_tf = inverted_index1[word].count(doc_id)/word_count[i]
    word_idf = math.log10(no_of_documents/len(inverted_index[word]))
    tf_idf_word[word][i] = word_tf*word_idf


In [11]:
len(inverted_index['chocol'])

2

In [12]:
tf_idf_word['chocol']

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.19621477, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.10791812, 0.        ])

In [0]:
def merging_doc(doc_id1,doc_id2,signal):
  count1 = 0
  count2 = 0
  merged_ids = []
  ## merge part
  while count1 < len(doc_id1) and count2 < len(doc_id2):
      if doc_id1[count1] < doc_id2[count2]:
        ## if 'OR' in query then only add this document id 
        ## else in intersection we only increment the counter
        if signal == 'union':
          merged_ids.append(doc_id1[count1])
        ## increment the counter of first document 
        count1 = count1 + 1
      elif doc_id1[count1] == doc_id2[count2]:
        ## if both the document ids are equal 
        ## then increment both the counters of the document ids 
        merged_ids.append(doc_id1[count1])
        count1 = count1 + 1
        count2 = count2 + 1
      else:
        ## if 'OR' in query then only add this document id 
        ## else in intersection we only increment the counter
        if signal == 'union':
          merged_ids.append(doc_id2[count2])
        ## increment the counter of second document 
        count2 = count2 + 1
  print("Merged them", merged_ids)
  return merged_ids

In [0]:
def merge_algorithm(query):


  tf_ifds_query = np.zeros((no_of_documents,))

## Lower the query part
  query = query.lower()
  ## Split the query
  query = query.split()
  query_splt = []
  for word in query:
    query_splt.append(ps.stem(word))

## Stemm the query part
  print("Stemmed_words " ,query_splt)
  store_qw_freq = {}


  sorted_terms = []
  ## If only 'AND' or 'OR' part is used in the query part
  ## Using the ------------------------Restrictive Model------------ in th 'IF' part
  ## creating word - len(posting) dictionary
  if ('and' not in query_splt) or ('or' not in query_splt):
    for i in range(len(query_splt)):
      if i%2 == 0:
        if query_splt[i] in inverted_index:
          
          store_qw_freq[query_splt[i]] = len(inverted_index[query_splt[i]])
    

## sorting the dictionary by value
    print(" Store qw freq", store_qw_freq)
    sorted_terms1 = sorted(store_qw_freq.items(), key=lambda kv: kv[1])

    len_sorted_terms = len(sorted_terms1)
    for l,values in enumerate(sorted_terms1):
      if ('and' not in query_splt):
        sorted_terms.append(values[0])
        if l != len_sorted_terms - 1:
          sorted_terms.append('or')
      if ('or' not in query_splt):
        sorted_terms.append(values[0])
        if l != len_sorted_terms - 1:
          sorted_terms.append('and')
    print("Sorted Terms",sorted_terms)
 ## If the query contains 'AND' and 'OR' part then go by sequence 
  else:
    for i in range(len(query_splt)):
        sorted_terms.append(query_splt[i])


  print("Sorted_terms" ,sorted_terms)

  merged_ids = []
  # count = 0
  doc_id1 = []
  doc_id2 = []

  ## Loop on all the terms of the query
  for i,word in enumerate(sorted_terms):
    ## if first word is empty then fill it 
    if doc_id1 == [] and (word != 'and' and word != 'or'):
      print ("Inside doc_id1 ",word)
    ## if word exist in inverted index
      if word in inverted_index:
        doc_id1 = inverted_index[word]
        print(word,"-", doc_id1)
    ## if second word exist is empty then fill it 
    elif doc_id2 == [] and (word != 'and' and word != 'or'):
      print ("Inside doc_id2 ",word)
      if word in inverted_index:
        doc_id2 = inverted_index[word]  
        print(word,"-", doc_id2)
    ## if first and second word are there then merge them
    if doc_id1 and doc_id2 and (word != 'and' and word != 'or'):
      ## use 'AND'
      if sorted_terms[i-1] == 'and':
        print("Taking Intersection", query_splt[i], query_splt[i-2])
        merged_ids = merging_doc(doc_id1,doc_id2,'intersection')
        ## Doing tf-idf part
        for d,document_id in enumerate(doc_ids):
          tf_ifds_query[d] = tf_idf_word[query_splt[i]][d]* tf_idf_word[query_splt[i - 2]][d]
      ## use 'OR'
      elif sorted_terms[i-1] == 'or':
        print("Taking Union", query_splt[i], query_splt[i-2])
        merged_ids = merging_doc(doc_id1,doc_id2,'union')
        for d,document_id in enumerate(doc_ids):
          tf_ifds_query[d] = tf_idf_word[query_splt[i]][d] + tf_idf_word[query_splt[i - 2]][d]
      doc_id1 = merged_ids
      doc_id2 = []
        
  print("Final Merged Ids",merged_ids)
  return tf_ifds_query

egg AND cheese

chocolate AND strawberry

eggs AND cheese AND bacon AND chocolate

chocolate OR strawberry OR eggs OR bacon

eggs AND cheese OR strawberry AND chocolate

(eggs AND cheese) OR (eggs AND bacon)



In [15]:
tf_idfs_query = merge_algorithm('eggs AND cheese OR strawberry AND chocolate or sum')

Stemmed_words  ['egg', 'and', 'chees', 'or', 'strawberri', 'and', 'chocol', 'or', 'sum']
Sorted_terms ['egg', 'and', 'chees', 'or', 'strawberri', 'and', 'chocol', 'or', 'sum']
Inside doc_id1  egg
egg - ['81503002321616896', '81587643376336896', '81673244926685184', '81736742478155778', '82650970722533376', '85032815321825280', '86441828815089664']
Inside doc_id2  chees
chees - ['81503002321616896', '81507775422791680', '81534165975171072', '81535634019323904', '81577509950459904', '81582996083326976', '81587643376336896', '81673244926685184', '81736742478155778', '82650970722533376', '85032815321825280', '85094773555335168', '86441828815089664']
Taking Intersection chees egg
Merged them ['81503002321616896', '81587643376336896', '81673244926685184', '81736742478155778', '82650970722533376', '85032815321825280', '86441828815089664']
Inside doc_id2  strawberri
strawberri - ['81500716438523904', '81623945064890368', '81656304107651072', '81715158593966080', '81716618236928000', '818423844

In [16]:
sum_of_tf_idf_doc_id = 0
## print tfids sum
for i,document_d in enumerate(doc_ids):
  print(document_d," - ", tf_idfs_query[i])
  sum_of_tf_idf_doc_id += tf_idfs_query[i]

print("\n\n\nSum of Tf Idf is ",sum_of_tf_idf_doc_id)

81499877556760576  -  0.0
81500716438523904  -  0.0
81503002321616896  -  0.0
81507775422791680  -  0.0
81534165975171072  -  0.0
81535634019323904  -  0.0
81577509950459904  -  0.0
81582996083326976  -  0.0
81587643376336896  -  0.0
81600113016971264  -  0.0
81623945064890368  -  0.0
81644157432643584  -  0.0
81656304107651072  -  0.19621477200865908
81673244926685184  -  0.0
81715158593966080  -  0.0
81716618236928000  -  0.0
81736742478155778  -  0.0
81842384404623360  -  0.0
81844590625304576  -  0.0
82461950231064576  -  0.0
82650970722533376  -  0.0
85032815321825280  -  0.0
85094773555335168  -  0.1079181246047625
86441828815089664  -  0.08118889657127093



Sum of Tf Idf is  0.3853217931846925


In [17]:
tf_idfs_query = merge_algorithm('eggs AND cheese')

Stemmed_words  ['egg', 'and', 'chees']
 Store qw freq {'egg': 7, 'chees': 13}
Sorted Terms ['egg', 'and', 'chees']
Sorted_terms ['egg', 'and', 'chees']
Inside doc_id1  egg
egg - ['81503002321616896', '81587643376336896', '81673244926685184', '81736742478155778', '82650970722533376', '85032815321825280', '86441828815089664']
Inside doc_id2  chees
chees - ['81503002321616896', '81507775422791680', '81534165975171072', '81535634019323904', '81577509950459904', '81582996083326976', '81587643376336896', '81673244926685184', '81736742478155778', '82650970722533376', '85032815321825280', '85094773555335168', '86441828815089664']
Taking Intersection chees egg
Merged them ['81503002321616896', '81587643376336896', '81673244926685184', '81736742478155778', '82650970722533376', '85032815321825280', '86441828815089664']
Final Merged Ids ['81503002321616896', '81587643376336896', '81673244926685184', '81736742478155778', '82650970722533376', '85032815321825280', '86441828815089664']


In [18]:
sum_of_tf_idf_doc_id = 0
## print tfids sum
for i,document_d in enumerate(doc_ids):
  print(document_d," - ", tf_idfs_query[i])
  sum_of_tf_idf_doc_id += tf_idfs_query[i]

print("\n\n\nSum of Tf Idf is ",sum_of_tf_idf_doc_id)

81499877556760576  -  0.0
81500716438523904  -  0.0
81503002321616896  -  0.0005565760265960224
81507775422791680  -  0.0
81534165975171072  -  0.0
81535634019323904  -  0.0
81577509950459904  -  0.0
81582996083326976  -  0.0
81587643376336896  -  0.0022263041063840896
81600113016971264  -  0.0
81623945064890368  -  0.0
81644157432643584  -  0.0
81656304107651072  -  0.0
81673244926685184  -  0.0011775492794097665
81715158593966080  -  0.0
81716618236928000  -  0.0
81736742478155778  -  0.003957873966905048
81842384404623360  -  0.0
81844590625304576  -  0.0
82461950231064576  -  0.0
82650970722533376  -  0.000726956442900927
85032815321825280  -  0.0006332598347048076
85094773555335168  -  0.0
86441828815089664  -  0.0004930223626594524



Sum of Tf Idf is  0.009771542019560113


In [19]:
tf_idfs_query = merge_algorithm('chocolate OR strawberry OR eggs OR bacon')

Stemmed_words  ['chocol', 'or', 'strawberri', 'or', 'egg', 'or', 'bacon']
 Store qw freq {'chocol': 2, 'strawberri': 10, 'egg': 7, 'bacon': 5}
Sorted Terms ['chocol', 'or', 'bacon', 'or', 'egg', 'or', 'strawberri']
Sorted_terms ['chocol', 'or', 'bacon', 'or', 'egg', 'or', 'strawberri']
Inside doc_id1  chocol
chocol - ['81656304107651072', '85094773555335168']
Inside doc_id2  bacon
bacon - ['81503002321616896', '81736742478155778', '82650970722533376', '85032815321825280', '86441828815089664']
Taking Union strawberri chocol
Merged them ['81503002321616896', '81656304107651072', '81736742478155778', '82650970722533376', '85032815321825280', '85094773555335168']
Inside doc_id2  egg
egg - ['81503002321616896', '81587643376336896', '81673244926685184', '81736742478155778', '82650970722533376', '85032815321825280', '86441828815089664']
Taking Union egg strawberri
Merged them ['81503002321616896', '81587643376336896', '81656304107651072', '81673244926685184', '81736742478155778', '82650970722

In [20]:
sum_of_tf_idf_doc_id = 0
## print tfids sum
for i,document_d in enumerate(doc_ids):
  print(document_d," - ", tf_idfs_query[i])
  sum_of_tf_idf_doc_id += tf_idfs_query[i]

print("\n\n\nSum of Tf Idf is ",sum_of_tf_idf_doc_id)

81499877556760576  -  0.0
81500716438523904  -  0.0
81503002321616896  -  0.07602215244205851
81507775422791680  -  0.0
81534165975171072  -  0.0
81535634019323904  -  0.0
81577509950459904  -  0.0
81582996083326976  -  0.0
81587643376336896  -  0.06688915021216864
81600113016971264  -  0.0
81623945064890368  -  0.0
81644157432643584  -  0.0
81656304107651072  -  0.0
81673244926685184  -  0.04864665469975901
81715158593966080  -  0.0
81716618236928000  -  0.0
81736742478155778  -  0.20272573984548936
81842384404623360  -  0.0
81844590625304576  -  0.0
82461950231064576  -  0.0
82650970722533376  -  0.08688245993378116
85032815321825280  -  0.08109029593819575
85094773555335168  -  0.0
86441828815089664  -  0.07155026112193744



Sum of Tf Idf is  0.6338067141933899


In [21]:
tf_idfs_query = merge_algorithm('eggs AND cheese OR strawberry AND chocolate')

Stemmed_words  ['egg', 'and', 'chees', 'or', 'strawberri', 'and', 'chocol']
Sorted_terms ['egg', 'and', 'chees', 'or', 'strawberri', 'and', 'chocol']
Inside doc_id1  egg
egg - ['81503002321616896', '81587643376336896', '81673244926685184', '81736742478155778', '82650970722533376', '85032815321825280', '86441828815089664']
Inside doc_id2  chees
chees - ['81503002321616896', '81507775422791680', '81534165975171072', '81535634019323904', '81577509950459904', '81582996083326976', '81587643376336896', '81673244926685184', '81736742478155778', '82650970722533376', '85032815321825280', '85094773555335168', '86441828815089664']
Taking Intersection chees egg
Merged them ['81503002321616896', '81587643376336896', '81673244926685184', '81736742478155778', '82650970722533376', '85032815321825280', '86441828815089664']
Inside doc_id2  strawberri
strawberri - ['81500716438523904', '81623945064890368', '81656304107651072', '81715158593966080', '81716618236928000', '81842384404623360', '81844590625304

In [22]:
sum_of_tf_idf_doc_id = 0
## print tfids sum
for i,document_d in enumerate(doc_ids):
  print(document_d," - ", tf_idfs_query[i])
  sum_of_tf_idf_doc_id += tf_idfs_query[i]

print("\n\n\nSum of Tf Idf is ",sum_of_tf_idf_doc_id)

81499877556760576  -  0.0
81500716438523904  -  0.0
81503002321616896  -  0.0
81507775422791680  -  0.0
81534165975171072  -  0.0
81535634019323904  -  0.0
81577509950459904  -  0.0
81582996083326976  -  0.0
81587643376336896  -  0.0
81600113016971264  -  0.0
81623945064890368  -  0.0
81644157432643584  -  0.0
81656304107651072  -  0.006782096555233813
81673244926685184  -  0.0
81715158593966080  -  0.0
81716618236928000  -  0.0
81736742478155778  -  0.0
81842384404623360  -  0.0
81844590625304576  -  0.0
82461950231064576  -  0.0
82650970722533376  -  0.0
85032815321825280  -  0.0
85094773555335168  -  0.0041031684159164574
86441828815089664  -  0.0



Sum of Tf Idf is  0.01088526497115027
